Generate data summary table

In [2]:
import pandas as pd
import numpy as np
import datetime
import os

In [3]:
frames = []

for file in os.listdir('spy_data'):
    if file[-4:] == '.csv':
        df = pd.read_csv('spy_data/' + file)
        df['quote_datetime'] = pd.to_datetime(df['quote_datetime'])
        df['expiration'] = pd.to_datetime(df['expiration'])
        df['quote_date'] = df['quote_datetime'][0].date()
        df['quote_date'] = pd.to_datetime(df['quote_date'])
        
        eod = datetime.datetime.combine(df['quote_datetime'][0].date(), datetime.time(16,0, 0))
        df = df.loc[df['quote_datetime'] == eod]
        
        df = df[['quote_date', 'expiration', 'strike', 'option_type', 'close', 'active_underlying_price', 'implied_volatility']]
        
        frames.append(df)

In [4]:
options = pd.concat(frames)

In [5]:
options['T'] = options['expiration'] - options['quote_date']
options['T'] = options['T'].dt.days
options['moneyness'] = options.apply(lambda row : row["active_underlying_price"] / row["strike"] if row["option_type"] == "C"
                                     else row["strike"] / row["active_underlying_price"], axis=1)

In [6]:
options = options.loc[(options['close']!=0)]

In [7]:
options.tail()

,quote_date,expiration,strike,option_type,close,active_underlying_price,implied_volatility,T,moneyness
15470,2023-10-31,2026-01-16,500.0,P,82.0,418.235,0.1808,808,1.195500
15510,2023-10-31,2026-01-16,550.0,P,132.7,418.235,0.2384,808,1.315050
15520,2023-10-31,2026-01-16,565.0,C,7.5,418.235,0.1390,808,0.740239
15550,2023-10-31,2026-01-16,600.0,P,183.1,418.235,0.2875,808,1.434600
15604,2023-10-31,2026-01-16,670.0,C,1.0,418.235,0.1323,808,0.624231


Categorize data into the following:

ITM, OTM, ATM
Calls vs Puts
TTM 1-20, 20-365, 365-1095 days

In [8]:
calls = options.loc[options['option_type']=='C']
calls_p1 = calls.loc[(1 < calls['T']) & (calls['T'] <= 20)]
calls_p2 = calls.loc[(20 < calls['T']) & (calls['T'] <= 365)]
calls_p3 = calls.loc[(365 < calls['T']) & (calls['T'] <= 1095)]
calls_atm = calls.loc[abs(calls['moneyness'] - 1) <= 0.01]
calls_itm = calls.loc[calls['moneyness'] - 1 > 0.01]
calls_otm = calls.loc[1 - calls['moneyness'] > 0.01]

puts = options.loc[options['option_type']=='P']
puts_p1 = puts.loc[(1 < puts['T']) & (puts['T'] <= 20)]
puts_p2 = puts.loc[(20 < puts['T']) & (puts['T'] <= 365)]
puts_p3 = puts.loc[(365 < puts['T']) & (puts['T'] <= 1095)]
puts_atm = puts.loc[abs(puts['moneyness'] - 1) <= 0.01]
puts_itm = puts.loc[puts['moneyness'] - 1 > 0.01]
puts_otm = puts.loc[1 - puts['moneyness'] > 0.01]

In [9]:
calls_p1_atm = pd.merge(calls_p1, calls_atm, how="inner")
calls_p2_atm = pd.merge(calls_p2, calls_atm, how="inner")
calls_p3_atm = pd.merge(calls_p3, calls_atm, how="inner")

calls_p1_itm = pd.merge(calls_p1, calls_itm, how="inner")
calls_p2_itm = pd.merge(calls_p2, calls_itm, how="inner")
calls_p3_itm = pd.merge(calls_p3, calls_itm, how="inner")

calls_p1_otm = pd.merge(calls_p1, calls_otm, how="inner")
calls_p2_otm = pd.merge(calls_p2, calls_otm, how="inner")
calls_p3_otm = pd.merge(calls_p3, calls_otm, how="inner")

puts_p1_atm = pd.merge(puts_p1, puts_atm, how="inner")
puts_p2_atm = pd.merge(puts_p2, puts_atm, how="inner")
puts_p3_atm = pd.merge(puts_p3, puts_atm, how="inner")

puts_p1_itm = pd.merge(puts_p1, puts_itm, how="inner")
puts_p2_itm = pd.merge(puts_p2, puts_itm, how="inner")
puts_p3_itm = pd.merge(puts_p3, puts_itm, how="inner")

puts_p1_otm = pd.merge(puts_p1, puts_otm, how="inner")
puts_p2_otm = pd.merge(puts_p2, puts_otm, how="inner")
puts_p3_otm = pd.merge(puts_p3, puts_otm, how="inner")

In [10]:
call_cats = np.mat(np.array([
    [calls, calls_atm, calls_itm, calls_otm],
    [calls_p1, calls_p1_atm, calls_p1_itm, calls_p1_otm],
    [calls_p2, calls_p2_atm, calls_p2_itm, calls_p2_otm],
    [calls_p3, calls_p3_atm, calls_p3_itm, calls_p3_otm]
]))
put_cats = np.mat(np.array([
    [puts, puts_atm, puts_itm, puts_otm],
    [puts_p1, puts_p1_atm, puts_p1_itm, puts_p1_otm],
    [puts_p2, puts_p2_atm, puts_p2_itm, puts_p2_otm],
    [puts_p3, puts_p3_atm, puts_p3_itm, puts_p3_otm]
]))

C:\Users\tejup\AppData\Local\Temp\ipykernel_11452\3903512000.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  call_cats = np.mat(np.array([
C:\Users\tejup\AppData\Local\Temp\ipykernel_11452\3903512000.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  put_cats = np.mat(np.array([


In [26]:
cols = ["Total", "ATM", "ITM", "OTM"]
rows = ["Total", "1-20 days", "20-365 days", "365-1095 days"]
veclen = np.vectorize(len)
vecmiv = np.vectorize(lambda x : np.mean(x['implied_volatility']))
vecmp = np.vectorize(lambda x : np.mean(x["close"]))

In [27]:
print(pd.DataFrame(np.round(veclen(call_cats), 2), columns=cols, index=rows))
print(pd.DataFrame(np.round(vecmiv(call_cats), 4), columns=cols, index=rows))
print(pd.DataFrame(np.round(vecmp(call_cats), 2), columns=cols, index=rows))

print(pd.DataFrame(np.round(veclen(put_cats), 2), columns=cols, index=rows))
print(pd.DataFrame(np.round(vecmiv(put_cats), 4), columns=cols, index=rows))
print(pd.DataFrame(np.round(vecmp(put_cats), 2), columns=cols, index=rows))

               Total   ATM    ITM    OTM
Total          36604  4341  10055  22208
1-20 days      11601  1822   3280   6499
20-365 days    19156  2034   4859  12263
365-1095 days   3475   151    961   2363
                Total     ATM     ITM     OTM
Total          0.2613  0.1813  0.4097  0.2097
1-20 days      0.1952  0.1664  0.2589  0.1711
20-365 days    0.1686  0.1696  0.2291  0.1445
365-1095 days  0.1795  0.1943  0.2362  0.1555
               Total    ATM     ITM    OTM
Total          15.96  11.81   42.44   4.77
1-20 days       7.29   4.71   22.25   0.46
20-365 days    16.83  16.74   47.24   4.80
365-1095 days  43.48  53.98  102.85  18.67
               Total   ATM    ITM    OTM
Total          44309  4427  11084  28798
1-20 days      15115  1839   3527   9749
20-365 days    22985  2093   5943  14949
365-1095 days   3467   160    857   2450
                Total     ATM     ITM     OTM
Total          0.3325  0.1819  0.3084  0.3650
1-20 days      0.2736  0.1665  0.2100  0.3168
20-365 